imports

pudl set up

In [1]:
# import the necessary packages
%load_ext autoreload
%autoreload 2

import logging
import os
import sys

import pandas as pd
import sqlalchemy as sa
import pudl

In [2]:
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)


read excel file from online

link: https://screeningtool.geoplatform.gov/en/downloads#3/33.47/-97.5

In [3]:
communities = pd.read_excel('https://static-data-screeningtool.geoplatform.gov/data-versions/1.0/data/score/downloadable/1.0-communities.xlsx')

geoprocessing prep

In [4]:
# get county ID
#make column a string
communities['Census tract 2010 ID'] = communities['Census tract 2010 ID'].astype(str)
# make sure census tract ID is 11 digits
communities['Census tract 2010 ID'] = communities['Census tract 2010 ID'].str.zfill(11)

#make county fips column
communities['county_id_fips'] = communities['Census tract 2010 ID'].str[0:5]

filter to disadvantaged

In [5]:
disadvantaged_communities = communities.loc[communities['Identified as disadvantaged']==True]

count how many tracts are in each county

In [6]:
num_of_dc = disadvantaged_communities['county_id_fips'].value_counts().reset_index()
num_of_dc = num_of_dc.rename(columns={'index':'county_id_fips','county_id_fips':'num_of_communities'})

connect to total amount

In [7]:
total_dc = communities['county_id_fips'].value_counts().reset_index()
total_dc = total_dc.rename(columns={'index':'county_id_fips','county_id_fips':'total_num_of_communities'})

In [8]:
community_complete = num_of_dc.merge(total_dc,on='county_id_fips',how='left')

In [9]:
community_complete['disadvantaged_percent_of_total_communities'] = community_complete['num_of_communities'] / community_complete['total_num_of_communities'] * 100
community_complete

,county_id_fips,num_of_communities,total_num_of_communities,disadvantaged_percent_of_total_communities
0,06037,1146,2346,48.849105
1,17031,611,1319,46.322972
2,36047,410,761,53.876478
3,48201,402,786,51.145038
4,26163,362,611,59.247136
...,...,...,...,...
2707,31151,1,4,25.000000
2708,31153,1,43,2.325581
2709,31161,1,2,50.000000
2710,31181,1,2,50.000000


get utility service territory from PUDL

In [10]:
eia_service_territory = pudl_out.service_territory_eia861()

/opt/homebrew/Caskroom/miniforge/base/envs/hub/lib/python3.10/site-packages/pudl/extract/eia861.py:32: UserWarning: Integration of EIA 861 into PUDL is still experimental and incomplete.
The data has not yet been validated, and the structure may change.
  warnings.warn(


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
The following reported NERC regions are not currently recognized and become         UNK values: []
The following reported NE

/opt/homebrew/Caskroom/miniforge/base/envs/hub/lib/python3.10/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


In [19]:
service_territory_2020 = eia_service_territory.query("report_date=='2020-01-01'")

In [21]:
test = community_complete.merge(service_territory_2020,on='county_id_fips')

In [23]:
j40 = test.groupby(['utility_name_eia','utility_id_eia','state']).agg({'num_of_communities':'sum','total_num_of_communities':'sum'}).reset_index()

In [25]:
j40 ['percent_disadvantaged'] = j40['num_of_communities'] / j40['total_num_of_communities'] * 100

j40

,utility_name_eia,utility_id_eia,state,num_of_communities,total_num_of_communities,percent_disadvantaged
0,"4 Rivers Electric Cooperative, Inc.",64196,KS,31,60,51.666667
1,4-County Electric Power Assn,6641,MS,33,45,73.333333
2,A & N Electric Coop,84,MD,5,8,62.500000
3,A & N Electric Coop,84,VA,7,16,43.750000
4,AEP Texas Central Company,3278,TX,438,599,73.121870
...,...,...,...,...,...,...
2728,Yampa Valley Electric Assn Inc,21081,WY,2,5,40.000000
2729,Yazoo Valley Elec Power Assn,21114,MS,27,29,93.103448
2730,Yellowstone Valley Elec Co-op,20997,MT,15,45,33.333333
2731,York Electric Coop Inc,21002,NC,92,298,30.872483


In [12]:
eia_service_territory

,county,short_form,state,utility_id_eia,utility_name_eia,report_date,state_id_fips,county_id_fips
0,Carroll,<NA>,IA,11581,City of Manning,2001-01-01,19,19027
1,Wright,<NA>,MO,11585,City of Mansfield,2001-01-01,29,29229
2,Bristol,<NA>,MA,11586,Town of Mansfield,2001-01-01,25,25005
3,Newton,<NA>,GA,11587,City of Mansfield,2001-01-01,13,13217
4,Sanpete,<NA>,UT,11588,City of Manti,2001-01-01,49,49039
...,...,...,...,...,...,...,...,...
235986,Neosho,True,KS,64196,"4 Rivers Electric Cooperative, Inc.",2020-01-01,20,20133
235987,Osage,True,KS,64196,"4 Rivers Electric Cooperative, Inc.",2020-01-01,20,20139
235988,Wabaunsee,True,KS,64196,"4 Rivers Electric Cooperative, Inc.",2020-01-01,20,20197
235989,Wilson,True,KS,64196,"4 Rivers Electric Cooperative, Inc.",2020-01-01,20,20205
